In [1]:
import numpy as np
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

from bs4 import BeautifulSoup

import datetime
import time

In [2]:
# Initializing empty lists
category, price, desc, rating, num_rating, discount, mrp, is_fassured, timestamp = [], [], [], [], [], [], [], [], []

#URLs
add = []

In [4]:
# Script to scrap home appliances by popularity
start = time.time()

url = "https://www.flipkart.com/home-kitchen/home-appliances/pr?sid=j9e,abm&p[]=facets.serviceability%5B%5D%3Dtrue&otracker=categorytree"
driver = webdriver.Chrome()
dcap = dict(DesiredCapabilities.CHROME)
dcap["chrome.page.settings.userAgent"] = ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.18362")
driver = webdriver.Chrome(desired_capabilities = dcap, service_args = ['--ignore-ssl-errors=true'])
driver.implicitly_wait(20)
driver.get(url)

# Fetching 50 pages from Home Appliances category
for i in range(1,51):
    
    driver.get(url)

    content = driver.page_source

    with open('../Home_Appliances/Home_Appliances_Data/Home_Appliances_' + str(i) + '.txt', 'w', encoding = 'utf-8') as f:
        f.write(content)
        f.close()
    
    soup = BeautifulSoup(content)

    for div in soup.find_all('div', attrs = {'class' : '_3liAhj _1R0K0g'}):
        pr = div.find('div', attrs = {'class' : '_1vC4OE'})
        des = div.find('a', attrs = {'class' : '_2cLu-l'})
        rt = div.find('div', attrs = {'class' : 'hGSR34'})
        no_rt = div.find('span', attrs = {'class', '_38sUEc'})
        dis = div.find('div', attrs = {'class' : 'VGWI6T'})
        mr = div.find('div', attrs = {'class' : '_3auQ3N'})
        fassured = div.find('div', attrs = {'class' : '_3LWrw9'})
    
        if pr == None:
            price.append(0)
        else:
            price.append(pr.text)
    
        if des == None:
            desc.append(0)
        else:
            desc.append(des['title'])
        
        if rt == None:
            rating.append(0)
        else:
            rating.append(rt.text)
        
        if no_rt == None:
            num_rating.append(0)
        else:
            num_rating.append(no_rt.text)
        
        if dis == None:
            discount.append(0)
        else:
            discount.append(dis.text)
        
        if mr == None:
            mrp.append(0)
        else:
            mrp.append(mr.text)
            
        if fassured == None:
            is_fassured.append(0)
        else:
            is_fassured.append(1)
        
        timestamp.append(datetime.datetime.now())
    
    # Using click() to navigate through home appliances pages

    add.append(url)
    next_link = driver.find_elements_by_class_name('_3fVaIS')
    next_link[-1].click()
    
    time.sleep(7)
    
    driver.implicitly_wait(20)
    url = driver.current_url
    
print('Time Taken : {} sec'.format(time.time() - start))

Time Taken : 582.8727750778198 sec


In [5]:
home = pd.DataFrame({
    'category' : np.repeat('Home Appliances', len(price)),
    'price' : price,
    'description' : desc,
    'rating' : rating,
    'num_ratings' : num_rating,
    'discount' : discount,
    'mrp' : mrp,
    'is_fassured' : is_fassured,
    'timestamp' : timestamp
})

In [6]:
home.to_csv('../Home_Appliances/Home_Appliances_CSVs/Home_Appliances.csv', index = False)